# This file is to take the .xlsx file created by reach_Curator_py38_v3.py script, and adds in the missing rows 
-  this file --> xlsx_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001_Grant_curate_20250421.xlsx"
-  saves out this file --> xlsx_save_path = rf"{main_path}\Grant_curate\manual_additions"

In [1]:
import pandas as pd
import numpy as np
import os
import shutil


## Modify these three paths as needed

In [11]:
root_path = r'G:\Grant\behavior_data\DLC_net' # change this to your root path where all the analysis folder live, i currently use is G:\Grant\behavior_data\DLC_net
root_folder = 'grant_reach10_swingDoor-christie-2025-04-21' # change this to your root folder for one specfic mouse (this should contain multiple sessions)

session_option = 'A'

### Create new session_option for each session

In [12]:
main_path = rf'{root_path}\{root_folder}'
print(f'main_path: {main_path}')

###### mouse session option A
if session_option == 'A':
    mouse = 'reach10'
    session_date = '20250421'
    session_ID = 'session001'


###### mouse session option B'
if session_option == 'B':
    mouse = 'reach10'
    session_date = '20250424'
    session_ID = 'session001'


    ###### mouse session option B'
if session_option == 'C':
    print("⚠️Info for sesion C does not exist yet⚠️")
    mouse = 'reach10'
    session_date = ' '
    session_ID = ' '

main_path: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21


In [13]:
if session_option == 'A':
    #xlsx_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001_Grant_curate_20250421.xlsx"
    xlsx_file = rf'{main_path}\Grant_curate\{session_date}_christielab_{session_ID}.xlsx'
    txt_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\20250421\christielab\session001\20250421_christielab_session001_events_shifted.txt"

if session_option == 'B':
    # Set path_frontCam and path_sideCam for session B manually
    #xlsx_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250424_christielab_session001_Grant_curate_20250421.xlsx"
    xlsx_file = rf'{main_path}\Grant_curate\{session_date}_christielab_{session_ID}.xlsx'
    txt_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\20250424\christielab\session001\20250424_christielab_session001_events_shifted.txt"


# Check if files exist
if not os.path.exists(xlsx_file):
    print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
if not os.path.exists(txt_file):
    print(f"⚠️Warning: {txt_file} does not exist⚠️")



In [14]:
# ---- Step 1: Make a backup in curator_backup folder
original_dir = os.path.dirname(xlsx_file)
backup_dir = os.path.join(original_dir, 'curator_direct_backup')
os.makedirs(backup_dir, exist_ok=True)

backup_file_path = os.path.join(backup_dir, os.path.basename(xlsx_file))
shutil.copy2(xlsx_file, backup_file_path)
print(f"✅ Backup saved to: {backup_file_path}")

# ---- (your parsing & merging code goes here)
# ... your code to build `combined` DataFrame ...



✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\curator_direct_backup\20250421_christielab_session001.xlsx


### This is the cell that takes the .xlsx file and the .txt file and creates a new .xlsx file with the manual additions
- makes a new folder called manual_additions if it does not exist
- reads the .xlsx file into a pandas dataframe
- reads the .txt file into a pandas dataframe
- creates a new dataframe with the manual additions
- saves the new dataframe to a new .xlsx file



In [15]:
# Load existing curated xlsx
df_orig = pd.read_excel(xlsx_file)
df_orig.head()

,reachInit,reachMax,reachEnd,stim,pellet_delivery,pellet_detected,T6000,T5000,behaviors
0,4423,4436,4449,0,4385,4308,4255,4388,grabbed
1,13669,13681,13696,0,13630,13477,13422,13632,grabbed
2,17666,17677,17681,0,17635,17259,17196,17637,dropped
3,26087,26098,26107,0,26061,25798,25742,26063,grabbed
4,30091,30101,30110,0,30065,29915,29857,30067,dropped


In [16]:

# Normalize column names just in case
df_orig.columns = df_orig.columns.str.strip()

# Load and parse txt as done earlier
events = []
current_event = {}
with open(txt_file, 'r') as f:
    for line in f:
        if line.strip():
            key, value = line.strip().split('\t')
            value = int(value)
            if key == 'T6000_played':
                if current_event:
                    events.append(current_event)
                current_event = {'T6000': value}
            elif key == 'pellet_detected':
                current_event['pellet_detected'] = value
            elif key == 'pellet_delivery':
                current_event['pellet_delivery'] = value
            elif key == 'T5000_played':
                current_event['T5000'] = value
if current_event:
    events.append(current_event)

events_df = pd.DataFrame(events)

# Find which T6000s are missing from original file
existing_T6000s = df_orig['T6000'].values
new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()

# Fill in missing columns to match structure
new_rows['reachInit'] = np.nan
new_rows['reachMax'] = np.nan
new_rows['reachEnd'] = np.nan
new_rows['stim'] = 0
new_rows['behaviors'] = np.nan

# # Reorder columns to match original file
# new_rows = new_rows[['reachinit', 'reachmax', 'reachend', 'stim',
#                      'pellet_delivery', 'pellet_detected', 't6000', 't5000', 'behaviors']]

# Reorder columns to match original file
new_rows = new_rows[['T6000', 'T5000', 'reachInit', 'reachMax',
                     'reachEnd', 'stim','behaviors' ,'pellet_delivery', 'pellet_detected']]

# Combine and sort by T6000
combined = pd.concat([df_orig, new_rows], ignore_index=True)
combined = combined.sort_values(by='T6000').reset_index(drop=True)

# Force final column order
combined = combined[['T6000', 'T5000', 'reachInit', 'reachMax',
                     'reachEnd', 'stim', 'behaviors',
                     'pellet_delivery', 'pellet_detected']]


# ---- Step 2: Save output as the same original file (overwrite it)
xlsx_save_path = xlsx_file
combined.to_excel(xlsx_save_path, index=False)
print(f"✅ Final merged Excel saved to: {xlsx_save_path}")


print(f"✅ Final merged CSV saved: {xlsx_save_path}")


✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx
✅ Final merged CSV saved: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx


In [17]:
# Load existing curated xlsx
df_modified = pd.read_excel(xlsx_save_path)
df_modified.head(10)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1768,2205.0,NaN,NaN,NaN,0,NaN,2203.0,1823.0
1,4255,4388.0,4423.0,4436.0,4449.0,0,grabbed,4385.0,4308.0
2,6582,6833.0,NaN,NaN,NaN,0,NaN,6831.0,6642.0
3,8895,9263.0,NaN,NaN,NaN,0,NaN,9261.0,8956.0
4,13422,13632.0,13669.0,13681.0,13696.0,0,grabbed,13630.0,13477.0
5,17196,17637.0,17666.0,17677.0,17681.0,0,dropped,17635.0,17259.0
6,21425,21573.0,NaN,NaN,NaN,0,NaN,21571.0,21492.0
7,25742,26063.0,26087.0,26098.0,26107.0,0,grabbed,26061.0,25798.0
8,29857,30067.0,30091.0,30101.0,30110.0,0,dropped,30065.0,29915.0
9,33286,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [18]:
df_orig.head(10)

,reachInit,reachMax,reachEnd,stim,pellet_delivery,pellet_detected,T6000,T5000,behaviors
0,4423,4436,4449,0,4385,4308,4255,4388,grabbed
1,13669,13681,13696,0,13630,13477,13422,13632,grabbed
2,17666,17677,17681,0,17635,17259,17196,17637,dropped
3,26087,26098,26107,0,26061,25798,25742,26063,grabbed
4,30091,30101,30110,0,30065,29915,29857,30067,dropped
5,35580,35590,35596,0,35558,35297,35240,35560,grabbed
6,43749,43759,43776,0,43724,43414,43356,43726,grabbed
7,52760,52798,52801,0,52714,52525,52468,52716,dropped
8,55245,55254,55261,0,55223,54842,54780,55226,grabbed
9,57664,57674,57679,0,57647,57345,57287,57650,missed
